# Seasonal Vegetation Anomalies



## Background

Understanding how the vegetated landscape responds to longer-term environmental drivers such as the El Nino Southern Oscillation (ENSO) or climate change, requires the calculation of seasonal anomalies. Seasonal anomalies subtract the long-term seasonal mean from a time-series, thus removing seasonal variability and highlighting change related to longer-term drivers. 

## Description

This notebook will calculate the seasonal anomaly for any given season and year. The long-term seasonal climatologies (both mean and standard deviation) for the vegetation index `NDVI` have been pre-calculated and are stored on disk. Given an AOI, season, and year, the script will calculate the seasonal mean for one of these indices and subtract the seasonal mean from the long-term climatology, resulting in a map of standardised vegetation anomalies for your AOI.  Optionally, the script will output a geotiff of the result. 

**IMPORTANT NOTES:** 

* It is a convention to establish climatologies based on a 30-year time range to account for inter-annual and inter-decadal modes of climate variability (often 1980-2010). As the landsat archive only goes back to 1987, the climatologies here have been calculated using the date-range `1988 - 2010` (inclusive).  While this is not ideal, a 22-year climatology should suffice to capture the bulk of inter-annual and inter-decadal variability, for example, both a major El Nino (1998) and a major La Nina (2010) are captured by this time-range.

* Files & scripts for running datacube stats to calculate vegetation climatologies are located here: `'/g/data/r78/cb3058/dea-notebooks/vegetation_anomlies/dcstats'`. 

* The pre-computed climatologies are stored here: `/g/data/r78/cb3058/dea-notebooks/vegetation_anomalies/results/NSW_NDVI_climatologies_<mean>`.  The script below will use this string location to grab the data, so shifting the climatology mosaics to another location will require editing the `anomalies.py` script.

* So far, NDVI climatolgies have been produced for the full extent of NSW only. 

## Technical details

* **Products used:** 'ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3'


## Getting Started

To run this analysis, go to the `Analysis Parameters` section and enter the relevant details, then run all the cells in the notebook. If running the analysis multiple times, only run the `Set up dask cluster` and `import libraries` cells once.

## Import libraries

In [26]:
import xarray as xr
from datacube.helpers import write_geotiff
import matplotlib.pyplot as plt
import geopandas as gpd
import sys
import os

sys.path.append('../Scripts')
from dea_plotting import display_map, map_shapefile
# from anomalies import calculate_anomalies, load_ard
from dea_dask import create_local_dask_cluster
from composites import calculate_composite, load_ard

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Set up local dask cluster

Dask will create a local cluster of cpus for running this analysis in parallel. If you'd like to see what the dask cluster is doing, click on the hyperlink that prints after you run the cell and you can watch the cluster run.

In [2]:
create_local_dask_cluster()

Client Scheduler: tcp://127.0.0.1:44844 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 8 Memory: 30.67 GB


## Analysis Parameters

The following cell sets the parameters, which define the area of interest and the season to conduct the analysis over. The parameters are:

* `shp_fpath`: Provide a filepath to a shapefile that defines your AOI, if not using a shapefile then put `None` here.
* `lat`, `lon`, `buffer`: If not using a shapefile to define the AOI, then use a latitide, longitude, and buffer to define a query 'box'.
* `collection`: The landsat collection to load data from. either `'c3'` or `'c2'`
* `year`: The year of interest, e.g. `'2018'`
* `season`:  The season of interest, e.g `'DJF'`,`'JFM'`, `'FMA'` etc
* `name` : A string value used to name the output geotiff, e.g 'NSW'
* `dask_chunks` : dictionary of values to chunk the data using dask e.g. `{'x':3000, 'y':3000}`

In [34]:
shp_fpath = "data/NSW_and_ACT.shp" #"data/nmdb_individual_catchments/PAROO RIVER.shp"
lat, lon, buff = -34.958, 150.281, 0.35 #.35
collection =  'c2'
year = '2020'
season = 'JFM'
month=2
name='NSW'
dask_chunks = {'x':3000, 'y':3000}


### Examine your area of interest

In [ ]:
# map_shapefile(gpd.read_file(shp_fpath), attribute='NSW_STAT_1')

In [4]:
display_map(y=(lat-buff, lat + buff), x=(lon-buff, lon + buff))

In [35]:
meta,comp = calculate_composite(shp_fpath=shp_fpath,
                                query_box=(lat,lon,buff),
                                collection=collection,
                                year=year,
                                season=season, 
                                month=None,
                                dask_chunks=dask_chunks)

print(meta)
print(comp)


DriverError: data/NSW_and_ACT.shp: No such file or directory

In [8]:
metadata=[ds.sizes,ds.time]
metadata

[Frozen(SortedKeysDict({'time': 8, 'y': 2873, 'x': 2474})),
 <xarray.DataArray 'time' (time: 8)>
 array(['2020-01-09T23:44:18.000000000', '2020-01-16T23:50:27.500000000',
        '2020-01-25T23:44:14.000000000', '2020-02-01T23:50:22.000000000',
        '2020-02-10T23:44:09.500000000', '2020-02-17T23:50:19.000000000',
        '2020-02-26T23:44:05.000000000', '2020-03-04T23:50:13.500000000'],
       dtype='datetime64[ns]')
 Coordinates:
   * time     (time) datetime64[ns] 2020-01-09T23:44:18 ... 2020-03-04T23:50:13.500000
 Attributes:
     units:    seconds since 1970-01-01 00:00:00]

## Calculate the anomaly for the AOI

For large queries (e.g > 10,000 x 10,000 pixels), the code will take several minutes to run.  Queries larger than ~25,000 x 25,000 pixels may start to fail due to memory limitations (several (42,000 x 35,000 x 52) runs covering all of NSW has been successfully run on the VDI). Check the x,y dimensions in the lazily loaded output to get idea of how big your result will be before you run  the `.compute()` cell.

In [5]:
#Lazily run calculations, this will check for errors before
# we actually compute the results
anomalies = calculate_anomalies(shp_fpath=shp_fpath,
                                query_box=(lat,lon,buff),
                                collection=collection,
                                year=year,
                                season=season,
                                dask_chunks=dask_chunks)

print(anomalies)

extracting data based on shapefile extent
loading Landsat C2
loading Landsat C2 pq data
making pq mask
start: 2020-01-02T00:32:10.000000000, end: 2020-02-23T00:07:14.000000000, time dim length: 33
calculating vegetation indice


RasterioIOError: results/NSW_NDVI_Climatologies_mean/mosaics/ndvi_clim_mean_JFM_nsw.tif: No such file or directory

In [ ]:
%%time
anomalies = anomalies.compute()

In [ ]:
anomalies

## Export geotiff

In [ ]:
# Write geotiff to a location
print('results'+name+'_ndvi_'+year+"_"+season+"_"+collection+ '_composite.tif')

In [ ]:
write_geotiff(name+'_ndvi_'+year+"_"+season+"_"+collection+ '_composite.tif', anomalies)

## Plot the result

If your AOI is very large, plotting the result can crash the notebook. In that case, its better to export the geotiff and view it in QGIS or ArcGIS

In [ ]:
anomalies.std_anomalies.plot(figsize=(10,10), vmin=-2.0, vmax=2.0, cmap='BrBG')

plt.title(season+ ", " +year)
plt.show()